<img src="https://www.ifsc.edu.br/image/layout_set_logo?img_id=1319584&t=1602803233260" width="20%">

<center>

---
# **Análise de Sentimento Multilíngue usando BERT**
## <u>Prof. Carlos Andres Ferrero</u>
## Instituto Federal de Santa Catarina (IFSC), Câmpus Lages
## Grupo de Pesquisa em Análise Inteligente de Dados (IDA-IFSC)
---
</center>

# Introdução

**Objetivo**: introduzir o uso do modelo BERT para construir um modelo de classificação multilíngue para análise de sentimento de *reviews* de aplicativos.

**Material e Método**: 

*Etapa 1 - Coleta de Dados:* os dados utilizados para contruir o modelo de classificação correspondem a avaliações de aplicativos, os quais foram extraídos utilizando webscrapping no site da Google Play Store. Cada avaliação possuia inicialmente um score de 1 a 5 e esses scores foram transformados em três classes ou labels: `postivo (score>3)`, `negativo (score<=3)`.

*Etapa 2 - Pré-processamento de Dados:* o atributo classe foi codificado e as avaliações tokenizadas (transformação em *tokens*) usando BertTokenizer. Nesta etapa são apresentados e explicados alguns conceitos e parâmetros usados na tokenização.

*Etapa 3 - Preparação dos Conjuntos de Dados:* o conjunto de dados com todas as avaliações é dividido em três conjuntos: treinamento, validação e teste.

*Etapa 4 - Definição do Modelo de Classificação:* um modelo para classificação de sentenças baseado em BERT é apresentado, bem como uma função para transformar as instâncias do problema em entradas (`input`) e saídas (`output`) corretos para o treinamento do modelo em questão.

*Etapa 5 - Treinamento do Modelo*: os conjuntos de dados treinamento e validação são utilizados para treinar o modelo, monitorar o treinamento e escolher o melhor modelo.

*Etapa 6 - Avaliação do Modelo*: os conjunto de teste é utilizado para avaliar o modelo de classificação com dados não observados durante o treinamento ou o monitoramento do treinamento. O modelo foi avaliado utilizando medida de acurácia.

**Escopo**:

O escopo deste documento é limitado a estudar análise de sentimento por meio de classificação, usando um dos modelos do estado da arte em Processamento de Linguagem Natural (NLP) e abstrai os conceitos de WordEmbeddings, Sentence Embeddings e estruturas internas do modelo de Rede Neural para classificação: como Recurrent Neural Networks e Attention Layer.

# Desenvolvimento

Instalação e importação de bibliotecas.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 8.2MB/s 
     |████████████████████████████████| 901kB 50.7MB/s 
     |████████████████████████████████| 3.3MB 49.1MB/s 


In [2]:
# Basic
import csv
import pandas as pd
import numpy as np
import os
import re

# Tensorflow
import tensorflow as tf

# BERT
from transformers import BertTokenizer, BasicTokenizer
from transformers import TFBertModel, TFBertPreTrainedModel, TFBertForSequenceClassification, BertConfig

# Scikit-leran
from sklearn.model_selection import train_test_split

## Etapa 1 - Coleta de Dados

As avaliações utilizadas neste trabalho foram coletadas utilizando as bibliotecas `selenium` e `google-play-scrapper`. O script em `data/app_scrapping` coleta o Top 10 de aplicativos gratuítos e pagos da plataforma Google Play Store nos paises US e BR. Essa coleta resultou em 40 apps e, como alguns apps são repetidos, o número de aplicativos únicos foi 34. Desses aplicativos foram obtidas avaliações/*reviews* em inglês e português, até 600 avaliações de cada, até 300 com rating de 1 a 3 (negativo) e até 300 com rating de 4 a 5 (positivas). Essa coleta resultou em 66751 avaliações, as quais são apresentadas a seguir.

In [3]:
df = pd.read_csv('reviews.csv')
df

,content,score,appId,lang,label
0,"I'm not giving 1 star because of the UI,I'm gi...",1,com.teamspeak.ts3client,en,negative
1,Waste of money. Don't buy this. Disconnects wh...,1,com.teamspeak.ts3client,en,negative
2,UI nightmare: - dialogs with checked checkbox ...,1,com.teamspeak.ts3client,en,negative
3,Super upset with how poorly this app performs ...,1,com.teamspeak.ts3client,en,negative
4,This app is rubbish pay for something that doe...,1,com.teamspeak.ts3client,en,negative
...,...,...,...,...,...
66747,"Sem dúvidas é o melhor launcher que tem, tanto...",5,com.teslacoilsw.launcher.prime,pt,positive
66748,Ótimo,5,com.teslacoilsw.launcher.prime,pt,positive
66749,"Melhor launcher de todos, disparado. Vale cada...",5,com.teslacoilsw.launcher.prime,pt,positive
66750,O MELHOR APP DO MUNDO!!,5,com.teslacoilsw.launcher.prime,pt,positive


Do total de avaliações, apenas 4 avaliações não tinham conetúdo algum.

In [4]:
df['content'].isna().sum()

4

Essas avaliações foram removidos do conjunto de dados, como segue.

In [5]:
df = df.dropna()
df

,content,score,appId,lang,label
0,"I'm not giving 1 star because of the UI,I'm gi...",1,com.teamspeak.ts3client,en,negative
1,Waste of money. Don't buy this. Disconnects wh...,1,com.teamspeak.ts3client,en,negative
2,UI nightmare: - dialogs with checked checkbox ...,1,com.teamspeak.ts3client,en,negative
3,Super upset with how poorly this app performs ...,1,com.teamspeak.ts3client,en,negative
4,This app is rubbish pay for something that doe...,1,com.teamspeak.ts3client,en,negative
...,...,...,...,...,...
66747,"Sem dúvidas é o melhor launcher que tem, tanto...",5,com.teslacoilsw.launcher.prime,pt,positive
66748,Ótimo,5,com.teslacoilsw.launcher.prime,pt,positive
66749,"Melhor launcher de todos, disparado. Vale cada...",5,com.teslacoilsw.launcher.prime,pt,positive
66750,O MELHOR APP DO MUNDO!!,5,com.teslacoilsw.launcher.prime,pt,positive


A seguir são apresentados o número de avaliações positivas e negativas, bem como a distribuição (em percentual) das instâncias do problema por classe.

In [6]:
df.label.value_counts()

positive    33773
negative    32975
Name: label, dtype: int64

In [7]:
df.label.value_counts(normalize=True).round(2)

positive    0.51
negative    0.49
Name: label, dtype: float64

A seguir são selecionadas as colunas de interesse `content` e `label`.

In [8]:
columns = ['content','label']
df = df[columns]

## Etapa 2 - Pré-processamento de Dados

Nesta etapa de pré-processamento inicialmente é realizada a codificação do atributo classe em um valor numérico, sendo `0` para negativo e `1` para positivo, bem a renomeação da coluna `content` para `text`.

In [9]:
to_replace = {
    'negative' : 0,    
    'positive' : 1    
}

df['label'].replace(to_replace, inplace=True)

to_rename = {
    'content' : 'text'
}

df.rename(columns=to_rename, inplace=True)

df

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text,label
0,"I'm not giving 1 star because of the UI,I'm gi...",0
1,Waste of money. Don't buy this. Disconnects wh...,0
2,UI nightmare: - dialogs with checked checkbox ...,0
3,Super upset with how poorly this app performs ...,0
4,This app is rubbish pay for something that doe...,0
...,...,...
66747,"Sem dúvidas é o melhor launcher que tem, tanto...",1
66748,Ótimo,1
66749,"Melhor launcher de todos, disparado. Vale cada...",1
66750,O MELHOR APP DO MUNDO!!,1


### Tokenização de Sentenças usando BertTokenizer

A tokenização é um processo importante da análise de textos, pois permite separar um texto em unidades menores, mais facilmente representáveis por métodos e técnicas de analise de dados.

In [10]:
PRETRAINED_MODEL = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL, do_lower_case=False)

Um exemplo de aplicação da tokenização usando BertTokenizer multilíngue, segue abaixo

In [11]:
sentence = "Not enjoying the app yet. Paid for full access for a year, and the WhatsApp feature isn't working properly"
sentence

"Not enjoying the app yet. Paid for full access for a year, and the WhatsApp feature isn't working properly"

In [12]:
tokenized_sentence = tokenizer(sentence)

for k, v in tokenized_sentence.items():
    print(k,v)

input_ids [101, 16040, 84874, 10230, 10105, 72894, 21833, 119, 107353, 10162, 10142, 13375, 18314, 10142, 169, 10924, 117, 10111, 10105, 12489, 10107, 10738, 16587, 19072, 98370, 112, 188, 14616, 83438, 102]
token_type_ids [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


O tokenizador BertTokenizer transforma cada sentença em três arrays de valores: `input_ids`, `token_type_ids` e `attention_mask`.
- `input_ids`: correspondem a índices de um dicionário (comumente denominado `vocab`) a que cada temo da sentença corresponde. Adicionalmente Bert adiciona alguns `ids` especiais, como de inicio e fim de sentença, bem como pontuações. Nesta versão do Bert, `bert-base-multilingual-cased`, temos um amplo vocabulário que inclui termos em mais de 100 idiomas e na forma case sensitive (palavras com maiúsculas ou minúsculas possuem `ids` diferentes).
- `token_type_ids`: em algumas aplicações Bert é utilizado em diálogos de pergunta resposta, como bots para conversa ou avaliação. No nosso estudo usamos apenas um sentença, portanto cada toke corresponde à mesma sentença.
- `attention_mask`: existem sentenças com poucos termos e outras com muitos termos. Em geral é necessário definir um tamanho máximo de termos para codificar e analisar, por exemplo os primeiros 10 tokens. Assim, sentenças com mais de 10 tokens são truncadas e, sentenças com menos, são preenchidas (`padding`) com `ids=0`. A informação de `attention_mask` indica quais `ids` devem ser considerados e quais não.

No exemplo abaixo, realizamos utilizandos as opções de número máximo de tokens para 10, utilizando as opções de `truncation` e `padding`:

In [13]:
sentences = ["WhatsApp feature isn't working properly","This is the best App","Not so bad"]
tokenized_sentences = tokenizer(sentences, max_length=10, padding=True, truncation=True)

for k, v in tokenized_sentences.items():
    print(k,v)

input_ids [[101, 12489, 10107, 10738, 16587, 19072, 98370, 112, 188, 102], [101, 10747, 10124, 10105, 12504, 73784, 102, 0, 0, 0], [101, 16040, 10380, 15838, 102, 0, 0, 0, 0, 0]]
token_type_ids [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention_mask [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]


Decodificação das sentenças pelos seus `input_ids`:

In [14]:
for i in range(len(sentences)):
    decoded = tokenizer.decode(tokenized_sentences['input_ids'][i])
    print("Original: {} | Decoded: {}".format(sentences[i], decoded) )    

Original: WhatsApp feature isn't working properly | Decoded: [CLS] WhatsApp feature isn't [SEP]
Original: This is the best App | Decoded: [CLS] This is the best App [SEP] [PAD] [PAD] [PAD]
Original: Not so bad | Decoded: [CLS] Not so bad [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]


Note que:
- Na sentença temos também token especiais que definem o inicio \[CLS\] e fim \[SEP\] da sentença, bem como de preenchimento \[PAD\].
- A primeira sentença foi truncada porque a trasformação em tokens resultou em mais do que o limite permitido de tokens.
- Nas sentenças 2 e 3 foi necessário realizar padding para preencher o espaço até 10 termos. Consequentmente os seus respectivos arrays `attention_mask` representam a informação de em quais tokens deve ter atenção e quais não.
As configurações de `max_length`, `padding` e `truncation`, são muito utilizadas no processo de tokenização de sentenças.

A seguir é apresentada uma função para codificar as nossas sentenças com configurações específicas, que será utilizada neste nosso trabalho.

In [15]:
def encode_sentences(X, tokenizer, max_length):
    return tokenizer.batch_encode_plus(X,
        max_length=max_length, # tamanho máximo da sequencia de tokens
        truncation=True, # truncar a sentença
        padding=True, # usar padding
        add_special_tokens=True, # adicionar tokens especiais [CLS] e [SEP]
        return_attention_mask=True, # Retornar attention_mask
        return_token_type_ids=False, # NÃO retornar token_type_ids, pois não são necessários        
        return_tensors='tf' # Formato para usar TensorFlow/Keras
        )

Execução da função `encode_sentences` para as três primeiras sentenças do conjunto de dados

In [16]:
sentences = df.text[:3]
print(sentences)

0    I'm not giving 1 star because of the UI,I'm gi...
1    Waste of money. Don't buy this. Disconnects wh...
2    UI nightmare: - dialogs with checked checkbox ...
Name: text, dtype: object


In [17]:
MAX_SEQUENCE_LENGTH = 64
encode_sentences(sentences, tokenizer, MAX_SEQUENCE_LENGTH )

{'input_ids': <tf.Tensor: shape=(3, 64), dtype=int32, numpy=
array([[   101,    146,    112,    181,  10472,  24426,    122,  16624,
         12373,  10108,  10105,    158,  11281,    117,    146,    112,
           181,  24426,    122,  16624,  10854,  10305,  10108,  11762,
         27874,  10107,    131,    122,    119,  10377,    112,    187,
        109513,  10376,  17920,  10105,  41008,  11674,  13246,  14884,
         12023,  10124,  13961,  10135,  11408,  22753,    119,    123,
           119,  11065,  10529,  10114,  19317,  20442,  12542,  38854,
           124,    119,  11065,  10529,  10114,  16868,    169,    102],
       [   101,  22034,  10216,  10108,  17920,    119,  11740,    112,
           188,  47715,  10531,    119,  42994,  15490,  39159,  10841,
           146,  31638,  10114,  12361,  10216,  16567,    119,    153,
         37026,    118,  10114,    118,  31311,  38199,  13663,  14884,
         15490,  17530,    119,  22319,  57026,  14722,  10708,  38854,
  

## Etapa 3 - Preparação dos Conjuntos de Dados

A divisão do conjunto de dados para poder treinar, monitorar o treinamento do modelo, e avaliá-o com novos dados, é uma tarefa importante no processo de indução de qualquer modelo. Neste trabalho dividimos o conjunto de dados em treino, validação e teste.

A seguir é apresentada a `train_val_test_split`, uma extensão de `train_test_split` com suporte a treino, validação e teste e, posteriormente uma explicação sobre o intuito de cada um desses três conjuntos.

In [18]:
from sklearn.model_selection import train_test_split

def train_val_test_split(X, y, val_size = 0.25, test_size = 0.25, random_state = 42, shuffle = True, stratify = None ):    
    X_train, X_aux, y_train, y_aux = \
        train_test_split(X, y, test_size = val_size + test_size, random_state=random_state, shuffle=shuffle, stratify=stratify)
    X_val, X_test, y_val, y_test = \
        train_test_split(X_aux, y_aux, test_size = test_size / (val_size + test_size), random_state=random_state, shuffle=shuffle,stratify=y_aux if stratify is not None else None)
    return X_train, X_val, X_test, y_train, y_val, y_test

A divisão do conjunto de dados em treino, validação e teste, ocorre da seguinte forma:
- `train` é usado para treinar o modelo de classificação
- `val` é usado como validação do modelo com novos durante o treinamento, a cada época. Isso permite monitorar o desempenho do algoritmo com novos dados, evitar possível overfitting, treinamento desnecessário e escolher um modelo com melhor desempenho para avaliação no futuro.
- `test` é usado, ao final, para avaliar o modelo escolhido com o conjunto de validação.

Neste trabalho será utilizada uma distribuição de 33% para treino 33% para validação e 34% para teste.

In [20]:
from sklearn.model_selection import train_test_split
X = df['text']
y = df['label']

val_size = 0.33
test_size = 0.34
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y, val_size=val_size, test_size=test_size, stratify=y)

Com os dados do conjunto de treinamento podemos estimar um valor adequado do parâmetro `MAX_SEQUENCE_LENGTH`, número máximo de tokens a serem considerado na análise. Neste estudo será considerado como MAX_SEQUENCE_LENGTH o valor que cobre 90\% dos textos do dataset.

In [21]:
train_sentences_tokenized = tokenizer.batch_encode_plus(X_train)

Token indices sequence length is longer than the specified maximum sequence length for this model (614 > 512). Running this sequence through the model will result in indexing errors


In [22]:
train_sentence_lengths = [ len(x) for x in train_sentences_tokenized['input_ids']]
print('90% das sentenças tem até {:.0f} tokens'.format( np.percentile(train_sentence_lengths, 90) ) )

90% das sentenças tem até 93 tokens


In [23]:
MAX_SEQUENCE_LENGTH = int( np.percentile(train_sentence_lengths, 90) )
MAX_SEQUENCE_LENGTH

93

Armazenamos o número de classes do problema para posterior utilização.

In [24]:
num_labels = len(y_train.unique()) # número de classes do problema

## Etapa 4 - Definição do Modelo de Classificação

Instanciamos o modelo BERT para classificação de sequências de tokens.

In [25]:
model = TFBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=num_labels, output_attentions=False, output_hidden_states=False)
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 177,854,978
Trainable params: 177,854,978
Non-trainable params: 0
_________________________________________________________________


O modelo de classificação BERT recebe como entrada um array com dois elementos. O primeiro elemento são os `input_ids` das sequências e o segundo elemento é a máscara de atenção (`attention_mask`) de cada sentença. 

Para facilitar o treainemtno do modelo definimos uma função que transforma valores de entrada e saída, X e y, para o formato esperado do modelo de classificação.

In [26]:
def transform_to_model_input(X, y):
    encoded_sentences = encode_sentences(X, tokenizer, MAX_SEQUENCE_LENGTH)
    X_ = [encoded_sentences['input_ids'], encoded_sentences['attention_mask']]
    y_ = tf.convert_to_tensor(y.values, dtype=tf.int16)    
    return X_, y_

Abaixo é realizada a transformação dos dados de treinamento e validação para o formato adequado a ser usado pela função de treinamento.

In [27]:
X_train_, y_train_ = transform_to_model_input(X_train, y_train)
X_val_  , y_val_  = transform_to_model_input(X_val  , y_val)

## Etapa 5 - Treinamento do Modelo

In [29]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [30]:
log_dir='tensorboard_data/tb_bert'
model_save_path='bert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True)]

BATCH_SIZE = 32
history = []
history_ = model.fit(X_train_, y_train_, batch_size=BATCH_SIZE, epochs=1, validation_data=(X_val_,y_val_), callbacks=callbacks)
history.append(history_)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [31]:
history_ = model.fit(X_train_, y_train_, batch_size=BATCH_SIZE, epochs=2, validation_data=(X_val_,y_val_), callbacks=callbacks)
history.append(history_)

Epoch 1/2
689/689 [==============================] - 582s 846ms/step - loss: 0.4185 - accuracy: 0.8133 - val_loss: 0.4527 - val_accuracy: 0.8024
Epoch 2/2
689/689 [==============================] - 582s 845ms/step - loss: 0.3380 - accuracy: 0.8570 - val_loss: 0.4518 - val_accuracy: 0.8046


## Etapa 6 - Avaliação do Modelo

In [32]:
best_model = TFBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=num_labels, output_attentions=False, output_hidden_states=False)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
best_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_save_path='bert_model.h5'
best_model.load_weights(model_save_path)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
X_test_ , y_test_  = transform_to_model_input(X_test, y_test)

In [34]:
BATCH_SIZE=32
loss_test, acc_test = best_model.evaluate(X_test_, y_test_, batch_size=BATCH_SIZE)

710/710 [==============================] - 150s 208ms/step - loss: 0.4590 - accuracy: 0.8030


In [35]:
print("acc_test:", acc_test)

acc_test: 0.8030403256416321


# Considerações

Neste estudo foi realizado o treinamento de um classificador baseado no modelo BERT multilingue. Foram utilizadas revisões dos aplicativos no Top 10 Gratuítos e Pagos nos Estados Unidos e no Brasil, tanto em idioma inglês quanto português. Os valores de acurácia do modelo construído para cada conjunto foram:
- Treino: 85.70%
- Validação: 80.46%
- Teste: 80.30%

Além disso, foi realizada a captura de dados de revisões de aplicativos no Top 10 Gratuitos e Pagos na França, em françês. E foi alcançada uma acurácia semelhante à encontrada no conjunto de Teste, de 80%, o que sustenta a ideia de que a representação do modelo BERT multilíngue.